# _Using Python to Access Hydrologic Data From Web Services_
### Author: Joseph Brewer w/ Dr. Jeff Horsburgh
### For: CEE 6110 - Hydroinformatics
> Description: This script, imported into a Jupyter Notebook, <br>
will explore the data available from the USGS Unit Values WaterOneFlow <br>
web service, retrieve a time series of 15-minute discharge values, <br>
calculate daily summarized data, and then visualize output. 

In [ ]:
import datetime
from suds.client import Client
from pandas import Series
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#### Connect to website 

In [2]:
now = datetime.datetime.now()
now1 = now.strftime("%Y-%m-%d %H:%M:%S")

#### Create inputs needed for the web service call. 

In [3]:
wsdlURL = 'http://hydroportal.cuahsi.org/nwisuv/cuahsi_1_1.asmx?WSDL'
siteCode = 'NWISUV:09315000'
variableCode = 'NWISUV:00060'
beginDate = '2018-08-01'
endDate = now1

#### Create a new object named NWIS for calling the web service 

In [ ]:
NWIS = Client(wsdlURL).service

###Call the GetValuesObject method to return the datavalues

In [ ]:
response = NWIS.GetValuesObject(siteCode, variableCode, beginDate, endDate)

###Get the site name from the response

In [ ]:
siteName = response.timeSeries[0].sourceInfo.siteName

###Load called objects into pandas series object
    

In [ ]:
a = []
b = []

###Get the values and load into the blank lists using append

In [ ]:
values = response.timeSeries[0].values[0].value

Loop through the values and load into the blank lists using append. 

In [ ]:
for v in values:
    a.append(float(v.value))
    b.append(v._dateTime)

Create a Pandas Series object from the lists. 
Set the index of the series object to the dates. 

In [ ]:
ts = Series(a, index=b)

Resample data on daily intercal with mean/min/max functions.  
Create hourly max, min, and avg series. 

In [ ]:
hourlyTotDisAvg = ts.resample(rule='1D', base=0).mean()
hourlyTotDisMax = ts.resample(rule='1D', base=0).max()
hourlyTotDisMin = ts.resample(rule='1D', base=0).min()

Plot Stuff
Use Matplotlib to create a plot of the time series streamflow statistics

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1, 1, 1)  

ts.plot(color='grey', linestyle='solid', label='15-minute streamflow values', alpha=0.5, linewidth=0.5)
hourlyTotDisAvg.plot(color='green', linestyle='solid', label='Daily avg flows',
                     marker = 'o', ms=2, linewidth =0.75)
hourlyTotDisMax.plot(color='red', linestyle='solid', label='Daily max flows',
                     marker = 'o', ms=2, linewidth =0.75)
hourlyTotDisMin.plot(color='blue', linestyle='solid', label='Daily min flows',
                     marker = 'o', ms=2, linewidth =0.75)

# Set some properties of the subplot to make it look nice
ax.set_ylabel('Discharge, cubic feet per second')
ax.set_xlabel('Date')
ax.grid(True)
ax.set_title('Daily Max, Min, & Avg Flows for: ' + siteName + ', ' + siteCode)
ax.set_xlim(beginDate, endDate)
legend = ax.legend(loc='upper left', shadow=True)
fig.autofmt_xdate()  # use auto-formatter to enable accurate date representation with mouse 
ax.xaxis.set_major_locator(mdates.DayLocator(interval=15)) # set ticks interval for every 15 days.

# Create a legend 
frame = legend.get_frame()
frame.set_facecolor('0.95')

# Create a frame around the legend.
for label in legend.get_texts():
    label.set_fontsize('large')
    
# Set the font size in the legend
for label in legend.get_lines():
    label.set_linewidth(1.5)
    



In [ ]:
plt.show()

In [ ]:
print('done')